In [1]:
# Import necessary libraries
from speedy_utils.all import *
import time
import random

DEBUG    | utils_print.py:324 (setup_logger) - Logging set to DEBUG


In [3]:
import random, time
from speedy_utils.multi_worker.thread import multi_thread   # ← your updated helper

# Task definitions ----------------------------------------------------
def light_task(x: int) -> int:
    """Simulate a network/file read that idles for ~0‑100 µs."""
    time.sleep(random.random() * 1e-4)
    return x

def heavy_task(x: int) -> int:
    """Pure‑Python CPU work that keeps the GIL for ~0.35 ms."""
    acc = 0
    for i in range(5_000):
        acc += (i * i) % 97
    return acc

# Test matrix ---------------------------------------------------------
scenarios = [
    ("small‑light",  1_000,  light_task),
    ("large‑light", 100_000, light_task),
    ("small‑heavy",  1_000,  heavy_task),
    ("large‑heavy", 10_000,  heavy_task),
]

results = []
for name, n_items, fn in scenarios:
    data = list(range(n_items))

    t0 = time.time()
    multi_thread(fn, data, progress=False)          # threaded run
    mt = time.time() - t0

    t0 = time.time()
    list(map(fn, data))                             # single‑thread run
    st = time.time() - t0

    results.append((name, n_items, mt, st, st / mt))

# Report --------------------------------------------------------------
try:
    from tabulate import tabulate
    header = ["scenario", "items", "multi_thread (s)",
                "single_thread (s)", "speed‑up×"]
    print(tabulate(results, headers=header,
                    floatfmt=("","", ".4f", ".4f", ".2f")))
except ImportError:      # fallback: plain text
    print("\n".join(
        f"{name:12} {n:7d}  MT={mt:.4f}s  ST={st:.4f}s  "
        f"speed‑up×={st/mt:.2f}"
        for name, n, mt, st, _ in results
    ))


[multi_thread] 768 items in 0.0620s
[multi_thread] 768 items in 0.0806s
[multi_thread] 768 items in 0.3746s
[multi_thread] 768 items in 0.5644s
scenario       items    multi_thread (s)    single_thread (s)    speed‑up×
-----------  -------  ------------------  -------------------  -----------
small‑light     1000              0.0628               0.1024         1.63
large‑light   100000              0.0822              10.2715       124.94
small‑heavy     1000              0.3755               0.3450         0.92
large‑heavy    10000              0.5660               3.4292         6.06
